# Initialisation

In [4]:
import os

import re
import sys
import time
import math
import random
import pickle
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import scipy.stats as stat 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.cm as cmap 
sns.set()

from pytz import timezone
from datetime import datetime
from sklearn import linear_model
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression 
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from skimage.io import imread
from pathlib import Path
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf

from sklearn.linear_model import LinearRegression

from torch.utils.tensorboard import SummaryWriter

def SaveVariable(Variable, FileName):
    '''
    Saved any variable to the local drive
    '''
    DirName = Path(FileName).parent.absolute()
    os.makedirs(DirName, exist_ok = True)

    with open(FileName, 'wb') as io:
        pickle.dump(Variable, io)
    
def LoadVariable(FileName):
    '''
    Loads a variable from the local drive
    '''
    with open(FileName, "rb") as io:
        Res = pickle.load(io)
    return Res

def NpShift(arr, num, fill_value = np.nan):
    '''
    Shifts a numpy array by 'num' places
    '''
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

def NxDtoN2xTxD(X_Data, T):
    '''
    Transforms an NxD matrix to an (N-T)xTxD tensor
    '''
    if T != 0:
        tmp = X_Data
        X_Data = X_Data.reshape(-1, 1, X_Data.shape[1])
        for t in range(T):
            X_Data = np.concatenate((NpShift(tmp, t+1)[:, np.newaxis, :], X_Data), axis = 1)
        if T > 0:
            Y_Data = X_Data[T:, -1, :]
            X_Data = X_Data[T:, :-1, :]
        else:
            Y_Data = X_Data[:T, -1, :] #T is already negative
            X_Data = X_Data[:T, :-1, :] #T is already negative
        return X_Data, Y_Data
    
    else:
        if (False): print("NxDtoN2xTxD:", "T [{T}] can't be zero")
        return X_Data, None

def GetAndShowShapes(SupervisedType, YisX, X, Y, X_Train, Y_Train, X_Test, Y_Test, TrainDataset, train_loader, RowsToShow = 5):
    '''
    Retrieves the relevant shapes from the dataset and if (False): prints them
    '''
    SupervisedType = SupervisedType.lower()
    
    if X_Train is not None:
        if (False): print(f"X_Train[:{RowsToShow}]:\n", X_Train[:RowsToShow])
    else:
        if (False): print(f"X[:{RowsToShow}]:\n", X[:RowsToShow])        
    if (False): print()
    
    # number of classes
    if Y is not None:
        K = len(set(Y)) if SupervisedType == "classification" and len(set(Y)) != 2 else (Y.shape[1] if SupervisedType == "multivariateregression" else 1) #An output_size (K) > 1 can be either Multiclass or Multivariate-Regression, like Lat/Lon coordinates
    elif Y_Train is not None:
        K = len(set(Y_Train)) if SupervisedType == "classification" and len(set(Y_Train)) != 2 else (Y_Train.shape[1] if SupervisedType == "multivariateregression" else 1)
    elif YisX == True:    # When we Forecast and data is NxD, only later to be transformed to NxTxD,
        if X is not None: # Y is the latest/newest row of X, i.e. Y comes from X, there is no Y.
            K = X.shape[1]
        elif X_Train is not None:
            K = X_Train.shape[1]
        
    if len(X_Train.shape) == 2:
        N, D = X_Train.shape if X_Train is not None else X.shape
        H1, W1 = (0, 0)
    elif len(X_Train.shape) == 3:
        D = 0
        N, H1, W1 = X_Train.shape if X_Train is not None else X.shape
    elif len(X_Train.shape) == 4:
        N, H1, W1, D = X_Train.shape if X_Train is not None else X.shape
    
    if X is not None and Y is not None:
        if (False): print("X.shape      ", X.shape, "Y.shape      ", Y.shape)
    elif X is not None:
        if (False): print("X.shape      ", X.shape)
    if X_Train is not None and Y_Train is not None:
        if (False): print("X_Train.shape", X_Train.shape, "Y_Train.shape", Y_Train.shape)
    elif X_Train is not None:
        if (False): print("X_Train.shape", X_Train.shape)
    if X_Test is not None and Y_Test is not None:
        if (False): print("X_Test.shape ", X_Test.shape, "Y_Test.shape ", Y_Test.shape)
    elif X_Test is not None:
        if (False): print("X_Test.shape ", X_Test.shape)
    if X is not None:
        if (False): print(f"X: min = {X.min():.4f}, max = {X.max():.4f}")
    if X_Train is not None:
        if (False): print(f"X_Train: min = {X_Train.min():.4f}, max = {X_Train.max():.4f}")
    if (False): print("K         ", K)
    if (False): print("N:", N, "H1:", H1, "W1:", W1, "D:", D)

    if TrainDataset is not None:
        if (False): print(f"\nTrainDataset length: {len(TrainDataset)}")
        if (False): print(f"Data after transformation with batch size = {batch_size}:")
        if train_loader is not None:
            tmpX, tmpY = next(iter(train_loader))
            if (False): print("X.shape", tmpX.shape, "\tY.shape", tmpY.shape)

        #Sanity check:
        assert np.allclose(TrainDataset.__getitem__(0)[0][0], X_Train[0])
    
    return K, N, D, H1, W1

def PlotEachVarInX(X_Series):
    '''
    Plots a Pandas Series variable
    '''
    X_Series.plot()
    plt.ylabel(X_Series.name)
    plt.show()

def PlotEachLagVarInX(X_Series, lag = 1):
    '''
    Plots a lag_plot of a Pandas Series variable
    '''
    lag_plot(X_Series, lag = lag)
    plt.title(X_Series.name)
    plt.show()
    
def Var_LagVar_Cor(X_Series, lag = 1):
    '''
    Returns the correlation of a Pandas Series with its lagged version
    '''
    tmp = pd.concat([X_Series.shift(lag), X_Series], axis = 1)
    tmp.columns = [f't{int(math.copysign(lag, lag) * (-1))}', 't']
    return tmp.corr()

def PlotAutocorrelation(X_Series):
    '''
    Plots the autocorrelation_plot from Pandas on a Pandas Series
    '''
    pd.plotting.autocorrelation_plot(X_Series)
    plt.title(X_Series.name)
    plt.show()

def PlotAutocorrelationLineplot(X_Series, MaxLag = 31):
    '''
    Plots an autocorrelation line plots
    '''
    plot_acf(X_Series, lags = MaxLag)
    plt.title(X_Series.name)
    plt.xlabel("lag")
    plt.ylabel("Autocorrelation")
    plt.show()

# Data

In [5]:
#Setting the root directory / working directory
path_root = f"{os.getcwd()}"
print(path_root)

C:\Users\31636\OneDrive\Desktop\Thesis Data


In [4]:
def GetData(T):
    #############################
    ### Data Hyperparameters ####
    CustomNAString = None #If missing values are encoded with a string, which is the string?
    #############################


    #################
    ### One File ###
    XY = pd.read_csv(f"{path_root}/Data/sb_data.csv", header = 0, parse_dates=[['Date', 'Hour']])
    XY["Date_Hour"] = pd.to_datetime(XY["Date_Hour"], format="%d/%m/%Y %H")
    XY = XY.set_index(["Date_Hour"]) #XY = XY.reset_index()
    #################

    
    ####################
    ### Handling NAs ###
    NBeforeNADrop = len(XY)
    XY = XY.dropna()
    DroppedNARows = NBeforeNADrop - len(XY)
    if DroppedNARows > 0: print("Dropped NA rows count:", DroppedNARows)

    if CustomNAString is not None:
        NBeforeCustomNADrop = len(XY)
        XY = XY.replace(CustomNAString, np.nan, regex = False).dropna()
        DroppedCustomNARows = NBeforeCustomNADrop - len(XY)
        if DroppedCustomNARows > 0: print("Dropped custom NA rows count:", DroppedCustomNARows)
    ####################


    ################################
    ### Getting a Train/Test set ###
    ColumnsToKeepX = ["Rented Bike Count"] #Columns to keep
    X = XY[ColumnsToKeepX].values.astype(float) #Keeping those columns

    N = X.shape[0] #N number of observations

    TrainPerc = 0.9 #Training set is 90% of the data
    X_Train = X[:int(N * TrainPerc)] #Creating Training set
    X_Test = X[int(N * TrainPerc):]  #Creating Test set
    ################################


    ########################
    ### Scaling the Data ###
    scaler = StandardScaler(with_mean = True, with_std = True)
    X_Train = scaler.fit_transform(X_Train)
    X_Test = scaler.transform(X_Test)
    ########################


    ###################################
    ### Transform from NxD to NxTxD ###
    X_Train, Y_Train = NxDtoN2xTxD(X_Train, T) #Transforming from NxD to NxTxD
    X_Test, Y_Test = NxDtoN2xTxD(X_Test, T) #Transforming from NxD to NxTxD
    ###################################


    #Getting the K number of outputs, NTrain number of observations on training set, and D dimensions of X input
    K, NTrain, D, _, _ = GetAndShowShapes("Regression", True, X, None, X_Train, Y_Train, X_Test, Y_Test, None, None, RowsToShow = 0)
    
    return(X_Train, X_Test, Y_Train, Y_Test, K, N, NTrain, D)

# Hyperparameter Tuning

In [5]:
lags = list(set([T+1 for T in range(50, 350)])) #Tuning T/lag from 51 to 350
usebiases = list(set([True, False])) #Tunning bias over TRUE and FALSE


#######################
### HyperParameters ###
#######################
HP_lags = {'Lag': lags} #Creating a dictionary for the "lags"
HP_usebiases = {'Bia': usebiases} #Creating a dictionary for the usebiases


HParamsList = [HP_lags, HP_usebiases] #Concatenating all the hyperparameters together so we can iterate over them


##############
### Metrics ##
##############
#Regression
MT_Loss = {"mse": 'MSE'} #Creating a dictionary for the loss which is the Mean Squared Error
MT_RMSE = {"rmse": 'RMSE'} #creating a dictionary for the Root Mean Squared Error metric
MT_R2 = {"r2": 'R2'} #Creating a dictionary for the R2 model metric

MetricsList = [MT_Loss, MT_RMSE, MT_R2] #Concatenating all the metrics together

In [6]:
def RunGridSearch(run_dir, hparams, metric_dict, CurLogsDir, session_num):
    with SummaryWriter(run_dir) as SummWriter: #Creating a Tensorboard writer to save the hyperparameters and results there
        print(hparams)
        
        X_Train, X_Test, Y_Train, Y_Test, K, N, NTrain, D = GetData(hparams["Lag"]) #Reading the data and transforming them with T (lag) being iterated over the lags hyperparameter
        Y_Train = Y_Train.squeeze()
        Y_Test = Y_Test.squeeze()
#         print(X_Train.shape)
#         print(X_Test.shape)
#         print(Y_Train.shape)
#         print(Y_Test.shape)
#         raise Exception("DebugStop!")
        start_time = time.time() #Keeping the time

        #Instantiating the model and training it
        model = LinearRegression(fit_intercept = hparams["Bia"], normalize = False).fit(X_Train.squeeze(), Y_Train)
        Pred = model.predict(X_Test.squeeze()) #Making predictions
#         print(Pred.shape)
#         raise Exception("DebugStop!")
        Targets = Y_Test.copy()
        Predictions = Pred.copy()

        # if "scaler" in locals() or "scaler" in globals():
        #     Targets = scaler.inverse_transform(Targets)
        #     Predictions = scaler.inverse_transform(Predictions)

        #Calculating the error
        d1 = Targets - Predictions
        d2 = Targets - Targets.mean()
        r2 = 1 - d1.dot(d1) / d2.dot(d2)
        MSE = d1.dot(d1) / len(Targets) #Mean Squared Error
        RMSE = np.sqrt(MSE) #Root Mean Squared Error
        
        fig = plt.figure() #Plotting the Actual and Forecasted values
        plt.plot(Targets, label = f"Actual")
        plt.plot(Predictions, label = f"Forecasted")
        plt.legend()
        
        elapsed_time = time.time() - start_time
        print(f"session_num {session_num}. r2: {r2:.2f},  RMSE: {RMSE:.4f}") #Printing the metrics
        
        SummWriter.add_figure("Actual VS Forecast", fig, global_step = session_num, close = True) #Adding the plot to tensorboard
        
        TestLossVec.append(MSE) #Updating the metrics locally too, for reference
        R2CVec.append(r2)                
        RMSEVec.append(RMSE)
        
        SummWriter.add_hparams(hparam_dict = hparams, metric_dict = {"Loss": MSE, "R2": r2, "RMSE": RMSE}, hparam_domain_discrete = None, run_name = str(session_num)) #Writing the hyperparameters and metrics on tensorboard.
        

In [7]:
#initialising the variables
session_num = 0
TestLossVec = []
R2CVec = []
RMSEVec = []

#Creating the logs directory
logdir = "C:/logs"
os.makedirs(logdir, exist_ok = True)
CurTime = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
CurLogsDir = logdir + "/" + CurTime + "/"
os.mkdir(CurLogsDir)
os.mkdir(CurLogsDir + "hparam_tuning")

#Running Grid-search
for lag in lags:
    for usebias in usebiases:
        hparams = dict(zip([list(item.keys())[0] for item in HParamsList], [lag, usebias])) #Creating this run's hyperparameters dictionary
        metrics = {list(KV.keys())[0]: KV[list(KV.keys())[0]] for KV in MetricsList} #Creating the metrics dictionary

        #Creating tensorboard directories
        os.mkdir(CurLogsDir + "run-" + str(session_num))
        os.mkdir(CurLogsDir + "run-" + str(session_num) + "/train")
        os.mkdir(CurLogsDir + "run-" + str(session_num) + "/train/plugins")
        os.mkdir(CurLogsDir + "run-" + str(session_num) + "/train/plugins/profile")
        os.mkdir(CurLogsDir + "run-" + str(session_num) + "/validation")

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        
        RunGridSearch(CurLogsDir + run_name, hparams, metrics, CurLogsDir, session_num)
        session_num += 1
print("ALL COMBINATIONS ARE NOW TRAINED!")

--- Starting trial: run-0
{'Lag': 51, 'Bia': False}
session_num 0. r2: 0.86,  RMSE: 0.2746
--- Starting trial: run-1
{'Lag': 51, 'Bia': True}
session_num 1. r2: 0.86,  RMSE: 0.2746
--- Starting trial: run-2
{'Lag': 52, 'Bia': False}
session_num 2. r2: 0.86,  RMSE: 0.2747
--- Starting trial: run-3
{'Lag': 52, 'Bia': True}
session_num 3. r2: 0.86,  RMSE: 0.2747
--- Starting trial: run-4
{'Lag': 53, 'Bia': False}
session_num 4. r2: 0.86,  RMSE: 0.2748
--- Starting trial: run-5
{'Lag': 53, 'Bia': True}
session_num 5. r2: 0.86,  RMSE: 0.2748
--- Starting trial: run-6
{'Lag': 54, 'Bia': False}
session_num 6. r2: 0.86,  RMSE: 0.2741
--- Starting trial: run-7
{'Lag': 54, 'Bia': True}
session_num 7. r2: 0.86,  RMSE: 0.2741
--- Starting trial: run-8
{'Lag': 55, 'Bia': False}
session_num 8. r2: 0.86,  RMSE: 0.2734
--- Starting trial: run-9
{'Lag': 55, 'Bia': True}
session_num 9. r2: 0.86,  RMSE: 0.2734
--- Starting trial: run-10
{'Lag': 56, 'Bia': False}
session_num 10. r2: 0.86,  RMSE: 0.2736
--

In [9]:
print(f"tensorboard --logdir={CurLogsDir}") #Run from the anaconda terminal

tensorboard --logdir=C:/logs/2021-06-21_01-17-58/


http://localhost:6006